In [5]:
from qiskit.quantum_info import Kraus, SuperOp
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit.providers.aer.noise import pauli_error
from qiskit.providers.aer.noise import depolarizing_error
from qiskit.providers.aer.noise import thermal_relaxation_error
from qiskit.providers.models import BackendProperties
from qiskit.providers.aer import noise

import numpy as np
from numpy import linalg
from qiskit.tools.monitor import job_monitor
from qiskit.algorithms.optimizers import SPSA, COBYLA
from qiskit import IBMQ, Aer

from qiskit.opflow import X, Y, Z, I
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.providers.basicaer import QasmSimulatorPy

In [2]:
#IBMQ.load_account()
#provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
#provider.backends()

In [22]:
nShots = 8192
J = -1
b = np.linspace(0,4,10) # add points
#optimizer = COBYLA(maxiter=50, tol=0.001, rhobeg=1.0)
optimizer = SPSA(maxiter=2000)
###############################
    
backend = Aer.get_backend("qasm_simulator")
quantum_instance = QuantumInstance(backend,
                                    shots=nShots)

intermediate_info = {
    'nfev': [],
    'parameters': [],
    'energy': [],
    'stddev': []
}

def callback(nfev, parameters, energy, stddev):
    intermediate_info['nfev'].append(nfev)
    intermediate_info['parameters'].append(parameters)
    intermediate_info['energy'].append(energy)
    intermediate_info['stddev'].append(stddev)
    
def Classical_GSenergy(H):
    E_l,P_l = linalg.eig(Operator(H).data)
    Egs = np.min(E_l)
    return Egs


In [23]:
num_qubits = 4
J=1
h=1
H = (Z^Z^I^I) + (I^Z^Z^I) + (I^I^Z^Z) + (Z^I^I^Z) + (Y^I^I^I) + (I^Y^I^I) + (I^I^Y^I) + (I^I^I^Y)


ansatz = EfficientSU2(num_qubits, reps=0, entanglement='linear', insert_barriers=True) 
np.random.seed(5)  # seed for reproducibility
initial_point = np.random.random(ansatz.num_parameters)
local_vqe = VQE(ansatz=ansatz,
                optimizer=optimizer,
                initial_point=initial_point,
                quantum_instance=QasmSimulatorPy(),
                callback=callback)

local_result = local_vqe.compute_minimum_eigenvalue(H)
print('Quantum GS energy  : ', local_result.eigenvalue)
print('Classical GS energy: ', Classical_GSenergy(H))

Quantum GS energy  :  (-5.08984375+0j)
Classical GS energy:  (-5.226251859505507-3.1530049635217123e-17j)


In [24]:
# implementazione hamiltoniana per qualisasi numero di siti N
# da ottimizzare

from qiskit.quantum_info.operators import Operator, Pauli

X = Operator(Pauli('X'))
Z = Operator(Pauli('Z'))
Y = Operator(Pauli('Y'))
I = Operator(Pauli('I'))

N=2
M=1
H = np.zeros((2**N,2**N), dtype='complex')
i=0
for k in range(N-1):
    A=1
    B=1
    while i<N:
        if i==k:
            A = np.kron(np.kron(A,Z),Z)
            B = np.kron(B,Z)
            B = np.kron(B,I)
            i += 2
        else:
            A = np.kron(A,I)
            B = np.kron(B,I)
            i+=1
        #print(i, np.shape(B)
        print(A)
    i = 0
    print(k)
    H = H + A


[[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  0.+0.j -0.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j -0.+0.j]
 [ 0.+0.j -0.+0.j -0.+0.j  1.-0.j]]
0


In [78]:
N=4
H = np.zeros((2**N,2**N), dtype='complex')

for i in range(N-1):
    k=0
    lista1 = []
    lista2 = []
    while k<N:
        if k==i:
            lista1.append(X)
            lista1.append(X)
            lista2.append(Z)
            lista2.append(I)
            k += 2
        else:
            lista1.append(I)
            lista2.append(I)
            k += 1
    #print(i, len(lista1),len(lista2))
    A=1
    B=1
    for j in range(N):
        A = np.kron(A,lista1[j])
        B = np.kron(B,lista2[j])
    #print(i, np.shape(A))
    H = H + A + B
    
lista1 = []
lista2 = []

#PBC
k = 0
while k<N:
        if k==0:
            lista1.append(X)
            lista2.append(I)
            k += 1
        elif k==N-1:
            lista1.append(X)
            lista2.append(Z)
            k += 1
        else:
            lista1.append(I)
            lista2.append(I)
            k += 1
A=1
B=1
for j in range(N):
    A = np.kron(A,lista1[j])
    B = np.kron(B,lista2[j])
H = H + A + B